In [27]:
!pip install fuzzywuzzy
!pip install keras_visualizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import numpy as np
import pandas as pd
import re
from fuzzywuzzy import fuzz
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import activations
from difflib import get_close_matches

In [29]:
# dictionaries 
en_letters = [' ','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
ru_letters = [' ','а','б','в','г','д','е','ё','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я']


In [30]:
# load data-set
data =  pd.read_csv('/content/crosswordsfiltered.csv')

data['ru'] = data['ru'].astype('str')
data['en'] = data['en'].astype('str')

data.describe()

,ru,en
count,711,711
unique,649,631
top,модель,partner
freq,3,3


In [31]:
# encode words to (3,) array for input chars and (1,) array  for output chars
def encode_dataset(lang_data,dic,target_lang_data,target_dic):
   data = []
   target = []
   for index,word in enumerate(lang_data):
      word = word.strip()
      target_word = target_lang_data[index]
      if len(target_word) > len(word) : 
         target_word = target_word[:len(word)]
      elif len(target_word) < len(word) : 
         word = word[:len(target_word)]

      target_word = target_word.strip()
      word = ' '+word.lower()+' '
      target_word = target_word.lower()

      for indx,ltr in enumerate(word):
            if ltr!=' ':
                row = [0] * 3
                for i,ltr in enumerate(word[(indx-1):(indx+2)]):
                    if ltr in dic:
                        row[i] =   dic.index(ltr)
                data.append(row) 
                target.append(target_dic.index(target_word[indx-1]))                    
   return data,target   

In [32]:

model = keras.models.Sequential([
    # layers.InputLayer(3,),  
    layers.Embedding(input_dim=34,output_dim=3,input_length=3), 
    layers.Flatten(),     
    layers.Dense(81,activation=activations.relu), 
    layers.Dense(27, activation="softmax")  
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 3)              102       
                                                                 
 flatten_1 (Flatten)         (None, 9)                 0         
                                                                 
 dense_2 (Dense)             (None, 81)                810       
                                                                 
 dense_3 (Dense)             (None, 27)                2214      
                                                                 
Total params: 3,126
Trainable params: 3,126
Non-trainable params: 0
_________________________________________________________________


In [33]:
# model config
model_loss = keras.losses.MeanAbsoluteError()
# model.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy())
model.compile(loss="categorical_crossentropy", optimizer= "adam", metrics=['accuracy'])

In [34]:
#data prepration / encoded 
X_train,Y_train = encode_dataset(data["ru"],ru_letters,data["en"],en_letters)
X_train = np.asarray(X_train)
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=27)
X_train

array([[ 0, 19,  6],
       [19,  6,  4],
       [ 6,  4, 14],
       ...,
       [19, 10, 19],
       [10, 19, 20],
       [19, 20,  0]])

In [35]:
# train model
model_history = model.fit(
    x=X_train,
    y=Y_train,
    epochs=1000,
    verbose=1
)

Epoch 1/1000
139/139 [==============================] - 2s 2ms/step - loss: 3.0378 - accuracy: 0.1820
Epoch 2/1000
139/139 [==============================] - 0s 2ms/step - loss: 2.2819 - accuracy: 0.4284
Epoch 3/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.8447 - accuracy: 0.5520
Epoch 4/1000
139/139 [==============================] - 0s 3ms/step - loss: 1.6439 - accuracy: 0.6345
Epoch 5/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.5238 - accuracy: 0.6818
Epoch 6/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.4438 - accuracy: 0.7141
Epoch 7/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.3859 - accuracy: 0.7278
Epoch 8/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.3440 - accuracy: 0.7398
Epoch 9/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.3111 - accuracy: 0.7423
Epoch 10/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.283

In [36]:
# evaluate model
model.evaluate(X_train, Y_train, verbose=1)

139/139 [==============================] - 0s 2ms/step - loss: 0.5111 - accuracy: 0.8309


[0.511106014251709, 0.8308906555175781]

In [37]:
# test model
x,_ = encode_dataset(["ветер"],ru_letters,['aaaaaaaaaaaaaaaaaaaaa'],en_letters)
predicted = model.predict(x)
output = ""
for c in predicted:
  output = output+  en_letters[list(c).index(max(c))]
output    

1/1 [==============================] - 0s 102ms/step


'veter'

In [38]:
# get closest matches
# галерея , фермер ,фигура,бар,план,порт,флаг,energy,карта,Паника,мотор     ----  белый - belly  ---- факультет ,,журнал
patterns = ['car', 'fish', 'belly', 'boy','girl','gallary','give','land','wind','water','','faculty','wind','winter','home','television','suit','store','journal','neighbor']

print('matched words:',get_close_matches(output,patterns))

matched words: ['water']
